In [180]:
class Patient:
    def __init__(self,iid, has_covid19: bool, arrival_time: int, boredom: int):
        self.id = iid
        self.has_covid19 = has_covid19
        self.arrival_time = arrival_time #Omadan to Bimarestan
        self.delivered_time = -1 #Raftan to Saf Doctor
        self.triage_time = -1
        self.inspection_time = -1
        self.finish_time = -1
        self.boredom = boredom
    
    def is_bored(self) -> bool:
        return False
    
    def __repr__(self):
        return "ID: " + str(self.id) + \
        ", Arrival Time: " + str(self.arrival_time) + ", Boredom: " + str(self.boredom) + \
        ", Has COVID19: " + str(self.has_covid19) + ", Delivered Time: " + str(self.delivered_time) + \
        ", Triage Time: " + str(self.triage_time) + ", Inspection Time: " + str(self.inspection_time) + \
        ", Finish Time: " + str(self.finish_time)

In [181]:
import queue

class HQueue:
    def __init__(self):
        self.covid19_queue = queue.Queue()
        self.others_queue = queue.Queue()
    
    def get(self) -> Patient:
        while not self.covid19_queue.empty():
            patient = self.covid19_queue.get()
            if not patient.is_bored():
                return patient
        while not self.others_queue.empty():
            patient = self.others_queue.get()
            if not patient.is_bored():
                return patient
        return None
    
    def put(self, patient: Type[Patient]):
        if patient.has_covid19:
            self.covid19_queue.put(patient)
        else:
            self.others_queue.put(patient)
    
    def size(self) -> int:
        return self.covid19_queue.qsize() + self.others_queue.qsize()

In [182]:
class MockInput:
    iid = 1
    def __init__(self, alpha, lam, myQueue):
        self.outQ = myQueue
        self.bordemAverage = alpha
        self.lam = lam
        
    def create_patients(self, size, lastArriaval):
        last_input = lastArriaval
        for i in range(size):
            has_covid19 = False
            if random.uniform() > 0.9:
                has_covid19 = True
            bordemTime = random.exponential(scale=self.bordemAverage)
            self.outQ.put(Patient(MockInput.iid, has_covid19=has_covid19, arrival_time=last_input, boredom=bordemTime))
            MockInput.iid += 1
            last_input += random.poisson(lam=self.lam)
        return last_input

In [183]:
from typing import List, Type
from numpy import random

class Triage:
    def __init__(self, miu: float, inQ: Type[HQueue], outQueues: List[queue.Queue]):
        self.miu = miu
        self.inQ = inQ
        self.outQueues = outQueues
        self.idleTime = 0
          
    def preferedQueue(self) -> Type[queue.Queue]:
        minQSize = 99999999999999999999999
        Q = []
        for q in self.outQueues:
            if q.qsize() < minQSize:
                minQSize = q.qsize()
                Q = q
        return Q
    
    def process(self, clock):
        patient = self.inQ.get()
        if patient != None:
            process = random.poisson(lam=self.miu)
            patient.triage_time = clock
            patient.delivered_time = clock + process
            print(patient)
            print("Tirage Idle time:", self.idleTime)
            Q = self.preferedQueue()
            Q.put(patient)
            return process
        return "ss"

In [184]:
class TriageWorkFlow:
    def __init__(self, alpha, lam, miu, outQueues: List[queue.Queue], maximum):
        self.myQueue = queue.Queue()
        self.mockInput = MockInput(alpha=alpha, lam=lam, myQueue=self.myQueue)
        self.triageQueue = HQueue()
        self.triage = Triage(miu, self.triageQueue, outQueues)
        self.maximum = maximum
        self.clock = 0
        self.outQueues = outQueues
        
    def process(self):
        numbers = 0
        step = 150
        lastArriaval = 0
        lastPatient = None
        while numbers < self.maximum:
            lastArriaval = self.mockInput.create_patients(step, lastArriaval)
            numbers += step
            while not self.myQueue.empty() or not lastPatient is None:
                if lastPatient is None:
                    lastPatient = self.myQueue.get()
                if lastPatient.arrival_time <= self.clock:
                    self.triageQueue.put(lastPatient)
                    lastPatient = None
                elif self.triageQueue.size() == 0:
                    self.triage.idleTime += lastPatient.arrival_time - self.clock
                    self.clock = lastPatient.arrival_time
                    self.triageQueue.put(lastPatient)
                    lastPatient = None
                self.clock += self.triage.process(self.clock)
        for q in self.outQueues:
            q.put("Tammam")
                
qs = []
qs.append(queue.Queue())
qs.append(queue.Queue())
qs.append(queue.Queue())
triageWorkFlow = TriageWorkFlow(alpha=1, lam=2.4, miu=1.1, outQueues=qs, maximum=500)
triageWorkFlow.process()
print("-------------")
print(qs[0].qsize())
print(qs[1].qsize())
print(qs[2].qsize())

ID: 1, Arrival Time: 0, Boredom: 1.5293966137925117, Has COVID19: True, Delivered Time: 2, Triage Time: 0, Inspection Time: -1, Finish Time: -1
Tirage Idle time: 0
ID: 2, Arrival Time: 4, Boredom: 0.968797115852689, Has COVID19: False, Delivered Time: 7, Triage Time: 4, Inspection Time: -1, Finish Time: -1
Tirage Idle time: 2
ID: 3, Arrival Time: 8, Boredom: 0.25601169214976083, Has COVID19: False, Delivered Time: 10, Triage Time: 8, Inspection Time: -1, Finish Time: -1
Tirage Idle time: 3
ID: 4, Arrival Time: 9, Boredom: 0.4275350095186974, Has COVID19: False, Delivered Time: 11, Triage Time: 10, Inspection Time: -1, Finish Time: -1
Tirage Idle time: 3
ID: 5, Arrival Time: 11, Boredom: 1.5765019411511445, Has COVID19: False, Delivered Time: 12, Triage Time: 11, Inspection Time: -1, Finish Time: -1
Tirage Idle time: 3
ID: 6, Arrival Time: 15, Boredom: 0.2148711403237248, Has COVID19: False, Delivered Time: 18, Triage Time: 15, Inspection Time: -1, Finish Time: -1
Tirage Idle time: 6
ID

In [185]:
class Doctor:
    def __init__(self, mean: float):
        self.mean = mean
        self.busy = False
        self.releaseTime = 0
        
        
    def process(self, patient, clock):
        self.busy = True
        patient.inspection_time = clock
        process = random.exponential(scale=self.mean)
        patient.finish_time = process + clock
        self.releaseTime = process + clock
        return process

In [188]:
class Room:
    def __init__(self, service_rates: List[float], inQ: Type[HQueue]):
        self.doctors = []
        self.inQ = inQ
        for rate in service_rates:
            self.doctors.append(Doctor(rate))
    
    def getDoctors(self) -> List[Doctor]:
        return self.doctors
    
    def is_all_busy(self):
        for doc in self.doctors:
            if not doc.busy:
                return False
        return True
    
    def find_nearest_time(self):
        mini = 999999999999999999999999999
        doctor = None
        for doc in self.doctors:
            if mini > doc.releaseTime:
                mini = doc.releaseTime
                doctor = doc
        return doctor
    
    def process(self, clock):
        patient = self.inQ.get()
        for doctor in self.doctors:
            if doctor.releaseTime <= clock:
                doctor.busy = False
                doctor.releaseTime = -1
        for doctor in self.doctors:
            if not doctor.busy:
                doctor.process(patient, clock)
        if self.is_all_busy():
            doctor = self.find_nearest_time()
            clock = doctor.releaseTime
            doctor.process(patient, clock)
            print(patient)
            return clock
        print(patient)
        return clock
            

In [189]:
class RoomWorkFlow:
    def __init__(self, service_rates: List[float], inQ: queue.Queue):
        self.roomQ = HQueue()
        self.room = Room(service_rates, self.roomQ)
        self.inQ = inQ
        self.clock = 0
    
    def process(self):
        patient = self.inQ.get()
        while patient != "Tammam":
            i = 0
            while patient.delivered_time <= self.clock:
                self.roomQ.put(patient)
                i += 1
                patient = self.inQ.get()
                if patient == "Tammam":
                    break
            if i == 0:
                self.clock = patient.delivered_time
            else:
                self.clock = self.room.process(self.clock)

service_rate = {2, 3}
roomWorkFlow = RoomWorkFlow([2, 3], qs[0])
roomWorkFlow.process()

ID: 1, Arrival Time: 0, Boredom: 1.5293966137925117, Has COVID19: True, Delivered Time: 2, Triage Time: 0, Inspection Time: 2.4503406748908527, Finish Time: 4.934322745347762
ID: 4, Arrival Time: 9, Boredom: 0.4275350095186974, Has COVID19: False, Delivered Time: 11, Triage Time: 10, Inspection Time: 12.101532424917268, Finish Time: 13.658994757065123
ID: 7, Arrival Time: 18, Boredom: 0.2562535194502462, Has COVID19: False, Delivered Time: 18, Triage Time: 18, Inspection Time: 19.622252182431552, Finish Time: 22.562912728290108
ID: 10, Arrival Time: 26, Boredom: 1.9319682870063704, Has COVID19: False, Delivered Time: 27, Triage Time: 26, Inspection Time: 28.130350025178338, Finish Time: 29.822222040877993
ID: 13, Arrival Time: 29, Boredom: 0.5964195694692637, Has COVID19: False, Delivered Time: 31, Triage Time: 29, Inspection Time: 33.69660800645377, Finish Time: 35.78407421709142
ID: 16, Arrival Time: 35, Boredom: 0.9301250556095408, Has COVID19: True, Delivered Time: 35, Triage Time:

In [168]:
print(triageWorkFlow.triage.idleTime)

770
